In [ ]:
## 1. Инициализация
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, count, desc, sum, to_date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [ ]:
# Создаем SparkSession
#spark = SparkSession.builder.appName("WebServerLogAnalysis").getOrCreate()
conf = SparkConf().setAppName("WebServerLogAnalysis").setMaster("local[*]")
spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [ ]:
## 2. Загрузка данных
schema = StructType([
    StructField("ip", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("method", StringType(), True),
    StructField("url", StringType(), True),
    StructField("response_code", IntegerType(), True),
    StructField("response_size", IntegerType(), True)
    ])

In [ ]:
# Путь к CSV-файлу
file_path = "web_server_logs.csv"

In [ ]:
# Загружаем данные из CSV
df = spark.read.csv(file_path, header=True, schema=schema)

In [ ]:
df.show()
df.printSchema()

In [ ]:
# Кэшируем DataFrame, так как он будет использоваться несколько раз
df.cache()

In [ ]:
# Показываем пример данных для проверки загрузки
print("Пример загруженных данных:")
df.show(5, truncate=False)

In [ ]:
# 3. Задачи анализа

In [ ]:
# Задача 1: Топ-10 активных IP-адресов
#Группируем данные по IP и подсчитываем количество запросов для каждого IP. Отображаем 10 наиболее активных IP.

In [ ]:
print("Топ-10 активных IP-адресов:")
top_ips = df.groupBy("ip").agg(count("*").alias("request_count")).orderBy(desc("request_count")).limit(10)
top_ips.show(truncate=False)

In [ ]:
# Задача 2: Количество запросов по HTTP-методам
# Группируем данные по HTTP-методу и подсчитываем количество запросов для каждого метода.

In [ ]:
print("Количество запросов по HTTP-методам:")

method_counts = df.groupBy("method").agg(count("*").alias("method_count")).orderBy(desc("method_count"))
# Отсортировано по количеству для типичного анализа\n",

method_counts.show(truncate=False)


In [ ]:
# Задача 3: Количество ответов с кодом 404
# Фильтруем и подсчитываем количество запросов с кодом ответа 404.

In [ ]:
count_404 = df.filter(col("response_code") == 404).count()
print(f"\nКоличество ответов с кодом 404: {count_404}")

In [ ]:
# Задача 4: Общий размер ответов по дням
#Группируем данные по дате и суммируем размеры ответов для каждого дня. Сортируем по дате.

In [ ]:
print("\nОбщий размер ответов по дням:")

# Преобразуем строку временной метки в дату.
# Временная метка от Faker имеет формат ISO, например, 'YYYY-MM-DDTHH:MM:SS' или 'YYYY-MM-DDTHH:MM:SS.ffffff'
# to_date() корректно извлекает часть с датой.
df_with_date = df.withColumn("date", to_date(col("timestamp")))
daily_response_size = df_with_date.groupBy("date").agg(sum("response_size").alias("total_response_size")).orderBy("date")

# Показываем результаты. show(truncate=False) по умолчанию отобразит до 20 строк.
# На изображении-примере было 17 строк; это совпадет, если уникальных дат <=20
# или если первые 17 показанных дат совпадают с примером.
daily_response_size.show(20, truncate=False)

In [ ]:
# 4. Остановка SparkSession
#Останавливаем SparkSession для освобождения ресурсов

In [ ]:
spark.stop()